In [ ]:
!pip install ultralytics
!pip install opencv-python


In [ ]:
!pip install moviepy

In [ ]:
import os
from ultralytics import YOLO
import cv2
from IPython.display import Image, display
from moviepy.editor import VideoFileClip


In [ ]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

In [ ]:
class_labels = model.names

In [ ]:
# Input video path
input_video_path = '/content/CodeAlpha_Object_Detection_Tracking/input_video.mp4'  
cap = cv2.VideoCapture(input_video_path)

# Check if the video was opened correctly
if not cap.isOpened():
    print(f"Error: Unable to open video '{input_video_path}'")
    exit()

# Output video path
output_dir = '/content/CodeAlpha_Object_Detection_Tracking/runs'
os.makedirs(output_dir, exist_ok=True)
output_video_path = os.path.join(output_dir, 'video.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

In [ ]:
# Read frames and perform detection and tracking
frame_count = 0
max_frames = 300  # Limit frames to avoid infinite loop

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects with YOLOv8
    results = model(frame)

    # Draw results on the frame
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Bounding box coordinates
        scores = result.boxes.conf.cpu().numpy()  # Confidence scores
        classes = result.boxes.cls.cpu().numpy()  # Predicted classes

        for box, score, cls in zip(boxes, scores, classes):
            x1, y1, x2, y2 = map(int, box)
            class_id = int(cls)
            label = f'{class_labels[class_id]} {score:.2f}'

            # Draw the detection box on the image
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Increase font size for the label text
            font_scale = 1.0
            font_thickness = 2
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
            text_x = x1
            text_y = y1 - 10

            # Ensure text stays within image boundaries
            if text_y - text_size[1] < 0:
                text_y = y1 + 10 + text_size[1]

            # Draw text background rectangle
            cv2.rectangle(frame, (text_x, text_y - text_size[1]), (text_x + text_size[0], text_y + 2), (0, 255, 0), cv2.FILLED)

            # Draw the label text on the image
            cv2.putText(frame, label, (text_x, text_y), font, font_scale, (0, 0, 0), font_thickness)

    # Write the processed frame to the output video
    out.write(frame)

    frame_count += 1

# Release the capture and close the output files
cap.release()
out.release()



In [ ]:
# Convertir la vidéo en gif
clip = VideoFileClip(output_video_path)
gif_path = os.path.join(output_dir, 'video.gif')
clip.write_gif(gif_path)

In [ ]:
display(Image(filename=gif_path))


In [ ]:
print(f"Traitement terminé. Sortie enregistrée dans '{output_video_path}' et GIF créé: '{gif_path}'")
